In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,1,27,69,65,Sandy,Millets,30,6,18,28-28
2,2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,4,35,58,43,Red,Paddy,37,2,16,DAP


In [4]:
cols=df.columns

In [5]:
for col in cols:
    print(col)
    print(df[col].unique().sum() )

id
281249625000
Temparature
441
Humidity
1403
Moisture
1845
Soil Type
ClayeySandyRedLoamyBlack
Crop Type
SugarcaneMilletsBarleyPaddyPulsesTobaccoGround NutsMaizeCottonWheatOil seeds
Nitrogen
897
Potassium
190
Phosphorous
903
Fertilizer Name
28-2817-17-1710-26-26DAP20-2014-35-14Urea


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               750000 non-null  int64 
 1   Temparature      750000 non-null  int64 
 2   Humidity         750000 non-null  int64 
 3   Moisture         750000 non-null  int64 
 4   Soil Type        750000 non-null  object
 5   Crop Type        750000 non-null  object
 6   Nitrogen         750000 non-null  int64 
 7   Potassium        750000 non-null  int64 
 8   Phosphorous      750000 non-null  int64 
 9   Fertilizer Name  750000 non-null  object
dtypes: int64(7), object(3)
memory usage: 57.2+ MB


In [7]:
df.sample(5)

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
531474,531474,25,67,28,Sandy,Ground Nuts,35,17,4,10-26-26
399634,399634,32,53,46,Clayey,Paddy,13,1,9,10-26-26
162408,162408,25,59,50,Loamy,Tobacco,16,16,5,DAP
591695,591695,38,52,53,Sandy,Barley,38,18,11,10-26-26
156275,156275,31,65,40,Loamy,Millets,21,4,23,20-20


In [8]:
df['Soil Type'].value_counts()

Soil Type
Sandy     156710
Black     150956
Clayey    148382
Red       148102
Loamy     145850
Name: count, dtype: int64

In [9]:
df['Fertilizer Name'].value_counts()

Fertilizer Name
14-35-14    114436
10-26-26    113887
17-17-17    112453
28-28       111158
20-20       110889
DAP          94860
Urea         92317
Name: count, dtype: int64

In [10]:
df['Crop Type'].value_counts()

Crop Type
Paddy          85754
Pulses         78180
Cotton         69171
Tobacco        68000
Wheat          66628
Millets        65291
Barley         65045
Sugarcane      64269
Oil seeds      64184
Maize          62569
Ground Nuts    60909
Name: count, dtype: int64

In [11]:
df=df.drop(['id'],axis=1)

In [12]:
df

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,27,69,65,Sandy,Millets,30,6,18,28-28
2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,35,58,43,Red,Paddy,37,2,16,DAP
...,...,...,...,...,...,...,...,...,...
749995,25,69,30,Clayey,Maize,8,16,6,28-28
749996,37,64,58,Loamy,Sugarcane,38,8,20,17-17-17
749997,35,68,59,Sandy,Ground Nuts,6,11,29,10-26-26
749998,31,68,29,Red,Cotton,9,11,12,20-20


In [13]:
from sklearn.preprocessing import LabelEncoder,StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
le=LabelEncoder()
df['Fertilizer Name']=le.fit_transform(df['Fertilizer Name'])

In [14]:
df.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,37,70,36,Clayey,Sugarcane,36,4,5,4
1,27,69,65,Sandy,Millets,30,6,18,4
2,29,63,32,Sandy,Millets,24,12,16,2
3,35,62,54,Sandy,Barley,39,12,4,0
4,35,58,43,Red,Paddy,37,2,16,5


In [15]:
X=df.drop(['Fertilizer Name'],axis=1)
y=df['Fertilizer Name']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
from sklearn.compose import ColumnTransformer
numerical=['Temparature', 'Humidity', 'Moisture','Nitrogen', 'Potassium', 'Phosphorous']
categorical=[ 'Soil Type', 'Crop Type']
target=['Fertilizer Name']

In [18]:
preprocessor= ColumnTransformer([
    ('num',StandardScaler(),numerical),
    ('cat',OneHotEncoder(drop='first'),categorical)

])

In [27]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", XGBClassifier(n_estimators = 300, max_depth = 9 , learning_rate = 0.3))
])


In [28]:
pipeline.fit(X_train, y_train)


,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [29]:
y_pred=pipeline.predict_proba(X_test)


In [30]:
y_pred

array([[0.05889639, 0.04136659, 0.26351932, ..., 0.15627933, 0.0420823 ,
        0.18465723],
       [0.22544666, 0.06870185, 0.0989098 , ..., 0.13857375, 0.13861327,
        0.09238075],
       [0.24536316, 0.14976391, 0.08884908, ..., 0.14541256, 0.1464052 ,
        0.1178543 ],
       ...,
       [0.25290188, 0.1122734 , 0.14707796, ..., 0.08319095, 0.07626158,
        0.24397418],
       [0.5080389 , 0.07139936, 0.0596187 , ..., 0.04431352, 0.09884058,
        0.05096531],
       [0.10700468, 0.20113029, 0.16475981, ..., 0.13620716, 0.11686721,
        0.03834872]], shape=(150000, 7), dtype=float32)

In [33]:
top_3_preds = np.argsort(y_pred, axis=1)[:, -3:][:, ::-1]

actual_labels = le.inverse_transform(y_test)
top_3_labels = np.array([le.inverse_transform(row) for row in top_3_preds])


In [34]:
actual_labels

array(['17-17-17', '14-35-14', '10-26-26', ..., '14-35-14', '10-26-26',
       '14-35-14'], shape=(150000,), dtype=object)

In [35]:
top_3_labels

array([['17-17-17', '20-20', 'Urea'],
       ['20-20', '10-26-26', 'DAP'],
       ['10-26-26', '14-35-14', 'DAP'],
       ...,
       ['10-26-26', 'Urea', '17-17-17'],
       ['10-26-26', '20-20', 'DAP'],
       ['20-20', '14-35-14', '17-17-17']], shape=(150000, 3), dtype=object)

In [36]:
def mapk(actual, predicted, k=3):
    def apk(a, p, k):
        score = 0.0
        for i in range(min(k, len(p))):
            if p[i] == a and a not in p[:i]:
                score += 1.0 / (i + 1)
        return score

    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

map3_score = mapk(actual_labels, top_3_labels.tolist(), k=3)
print(f"MAP@3 Score: {map3_score:.5f}")

MAP@3 Score: 0.32583


In [37]:
test=pd.read_csv('test.csv')

In [55]:
result=test

In [38]:
test=test.drop(['id'],axis=1)

In [39]:
y_test_pred=pipeline.predict_proba(test)

In [40]:
y_test_pred

array([[0.11309048, 0.05081555, 0.15175664, ..., 0.13914034, 0.12663302,
        0.1721669 ],
       [0.1564923 , 0.03909702, 0.3443261 , ..., 0.06158494, 0.07993019,
        0.1839895 ],
       [0.11282259, 0.07205828, 0.052744  , ..., 0.22147393, 0.05421787,
        0.27063805],
       ...,
       [0.2057097 , 0.15088075, 0.05713794, ..., 0.09003848, 0.19990869,
        0.1512397 ],
       [0.14529648, 0.14899626, 0.14007698, ..., 0.25675955, 0.17321657,
        0.08486734],
       [0.1094522 , 0.303023  , 0.2567116 , ..., 0.06651761, 0.05187904,
        0.07037458]], shape=(250000, 7), dtype=float32)

In [42]:
top_3_pred_test = np.argsort(y_test_pred, axis=1)[:, -3:][:, ::-1]
top_3_labels = np.array([le.inverse_transform(row) for row in top_3_pred_test])


In [43]:
top_3_labels

array([['20-20', 'Urea', '17-17-17'],
       ['17-17-17', 'Urea', '10-26-26'],
       ['Urea', '28-28', '20-20'],
       ...,
       ['10-26-26', 'DAP', 'Urea'],
       ['28-28', 'DAP', '14-35-14'],
       ['14-35-14', '17-17-17', '20-20']], shape=(250000, 3), dtype=object)

In [44]:
df=pd.DataFrame(top_3_labels)

In [46]:
df['Fertilizer Name'] = df.iloc[:, 0] + ' ' + df.iloc[:, 1] + ' ' + df.iloc[:, 2]


In [58]:
result = pd.concat([result, pd.DataFrame(df, columns=['Fertilizer Name'])], axis=1)


In [59]:
result

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,750000,31,70,52,Sandy,Wheat,34,11,24,20-20 Urea 17-17-17
1,750001,27,62,45,Red,Sugarcane,30,14,15,17-17-17 Urea 10-26-26
2,750002,28,72,28,Clayey,Ground Nuts,14,15,4,Urea 28-28 20-20
3,750003,37,53,57,Black,Ground Nuts,18,17,36,DAP 14-35-14 Urea
4,750004,31,55,32,Red,Pulses,13,19,14,20-20 Urea 10-26-26
...,...,...,...,...,...,...,...,...,...,...
249995,999995,26,66,30,Red,Sugarcane,14,7,18,Urea 28-28 14-35-14
249996,999996,33,62,55,Red,Pulses,28,14,7,10-26-26 17-17-17 14-35-14
249997,999997,36,53,64,Black,Paddy,28,11,27,10-26-26 DAP Urea
249998,999998,36,67,26,Clayey,Paddy,33,0,10,28-28 DAP 14-35-14


In [60]:
submission=result[['id','Fertilizer Name']]
submission

,id,Fertilizer Name
0,750000,20-20 Urea 17-17-17
1,750001,17-17-17 Urea 10-26-26
2,750002,Urea 28-28 20-20
3,750003,DAP 14-35-14 Urea
4,750004,20-20 Urea 10-26-26
...,...,...
249995,999995,Urea 28-28 14-35-14
249996,999996,10-26-26 17-17-17 14-35-14
249997,999997,10-26-26 DAP Urea
249998,999998,28-28 DAP 14-35-14


In [61]:
submission.to_csv('submission.csv',index=False)